In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras import utils 
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import tensorflow_hub as hub
# Setting random seeds to reduce the amount of randomness in the neural net weights and results
# The results may still not be exactly reproducible
np.random.seed(42)
tf.random.set_seed(42)

2023-11-01 14:17:36.945057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 14:17:37.145051: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-01 14:17:37.909094: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/larkinan/miniconda3/envs/tf/lib/python3.9/site-packages/cv2/../../lib

In [2]:
# Testing to ensure GPU is being utilized
# Ensure that the Runtime Type for this notebook is set to GPU
# If a GPU device is not found, change the runtime type under:
# Runtime>> Change runtime type>> Hardware accelerator>> GPU
# and run the notebook from the beginning again.

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-11-01 14:17:38.956491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 14:17:39.063465: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 14:17:39.063503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 14:17:39.206879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 14:17:39.207148: I tensorflow/compile

In [3]:
BATCH_SIZE = 64

def convertCodeToNumpy(code):
    encoding = []
    for charIndex in range(len(code)):
        curCar = code[charIndex]
        if curCar =='0':
            encoding.append(0)
        elif curCar =='1':
            encoding.append(1)
    return(encoding)

In [4]:
# Defining a function to read the image, decode the image from given tensor and one-hot encode the image label class.
# Changing the channels para in tf.io.decode_jpeg from 3 to 1 changes the output images from RGB coloured to grayscale.
num_classes = 7
def _parse_function(filename):   
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)    # channels=1 to convert to grayscale, channels=3 to convert to RGB.
    #image_decoded = tf.image.resize(image_decoded, [224, 224])
    image_decoded = tf.cast(image_decoded, dtype=tf.float32)
    
    return(image_decoded)

In [5]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5",
               trainable=False, arguments=dict(batch_norm_momentum=0.997)),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(7,activation='sigmoid')
])
model.compile(
    loss=[tf.keras.losses.BinaryCrossentropy(from_logits=True)],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])#,run_eagerly=True)
weightFile = "/mnt/h/Aspire/BERT/place/place_image_model_checkpoint.h5"
checkpoint = ModelCheckpoint(filepath=weightFile,
                                monitor='val_loss',
                                save_best_only=True,
                                save_weights_only=True,
                                verbose=1,
                                initial_value_threshold=  None#0.97828
                                )
model(np.zeros((1,224,224,3)))
if(os.path.exists(weightFile)):
    model.load_weights(weightFile)

2023-11-01 14:17:39.327210: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 14:17:39.327278: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 14:17:39.327296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 14:17:39.327447: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 14:17:39.327466: I tensorflow/compile

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2023-11-01 14:17:41.867888: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2023-11-01 14:17:42.659141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [7]:

def mapOrigFilepath(filename):
    return filename[:-11] + ".jpg"

def loadImgs(imgFilepaths):
    arr = list(map(_parse_function,imgFilepaths))
    arr = np.asarray(arr)
    return arr

def prepFilepaths(foldername):
    
    folderToProcess = '/mnt/e/ASP3IRE/testPad/' + foldername
    
    def mapFullFilepath(filename):
        return folderToProcess + '/' + filename
    
    filenames = os.listdir(folderToProcess)
    filesToProcess = list(map(mapFullFilepath,filenames))
    origFilenames = list(map(mapOrigFilepath,filenames))
    return([filesToProcess,origFilenames])

def createCSV(foldername,predictions,origFilenames):
    df = pd.DataFrame(data = predictions,  
                  columns = ['isDaycare','isPark','isHome','isSchool','isNeighborhood','isIndoor','isOutdoor'])
    df['filename'] = origFilenames
    df.to_csv('/mnt/h/Aspire/PredStore/image_place_pred/' + foldername + ".csv",index=False)

def processSingleFolder(foldername):
    outputFilename = '/mnt/h/Aspire/PredStore/image_place_pred/' + foldername + '.csv'
    if (os.path.exists(outputFilename)):
        print("%s already exists" %(outputFilename))
        return
    filesToProcess,origFilenames = prepFilepaths(foldername)
    imgs = loadImgs(filesToProcess)
    preds = model.predict(imgs)
    createCSV(foldername,preds,origFilenames)

In [8]:
foldersToProcess = os.listdir('/mnt/e/ASP3IRE/testPad')
for folder in foldersToProcess:
    processSingleFolder(str(folder))

402/402 [==============================] - 8s 21ms/step


2023-11-01 14:54:48.387263: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 588.0KiB (rounded to 602112)requested by op Cast
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-11-01 14:54:48.391465: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-11-01 14:54:48.391480: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 48, Chunks in use: 48. 12.0KiB allocated for chunks. 12.0KiB in use in bin. 7.4KiB client-requested in use in bin.
2023-11-01 14:54:48.391485: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 34, Chunks in use: 33. 17.8KiB allocated for chunks. 17.2KiB in use in bin. 16.4KiB client-requested in use in bin.
2023-11-01 14:54:48.391488: I tensorflow/tsl/framework/bfc_a

ResourceExhaustedError: {{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Cast]